Trying out basic commands according to https://www.kaggle.com/apapiu/house-prices-advanced-regression-techniques/regularized-linear-models/comments, on top of which, model selection with LASSO
http://scikit-learn.org/stable/auto_examples/feature_selection/plot_select_from_model_boston.html#sphx-glr-auto-examples-feature-selection-plot-select-from-model-boston-py

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
train.shape, test.shape

In [ ]:
#train.describe()
#train.info()

**n = 1460 samples**


**d = 79 features** 80 columns: 1st ID 80th SalePrice (Y)


Some contain lots of nulls such as 
PoolQC           7 non-null object

Non-numerical features about half

**Goal: predict SalePrice** 
Predict a quantity with <100K samples 

In [ ]:
#train["SalePrice"].hist();

In [ ]:
#prices = np.log1p(train["SalePrice"])
#prices.hist()

In [ ]:
train["SalePrice"] = np.log1p(train["SalePrice"])
y_train = train["SalePrice"]

In [ ]:
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

In [ ]:
from scipy.stats import skew
from scipy.stats.stats import pearsonr
#log transform skewed numeric features:
numeric_feats = train.dtypes[train.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

In [ ]:
'SalePrice' in skewed_feats

In [ ]:
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])
#Convert categorical variable into dummy/indicator variables
all_data = pd.get_dummies(all_data)
all_data = all_data.fillna(all_data.mean())
all_data.shape

In [ ]:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
X_train.shape, X_test.shape

In [ ]:
#X_train.info

1. Random split into training and test sets.

  [1]: http://scikit-learn.org/stable/modules/cross_validation.html
  [2]: http://scikit-learn.org/stable/modules/linear_model.html#ridge-regression

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(train_data, train_target, test_size=0.4, random_state=0)
#X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

#reg = linear_model.Ridge(alpha = .5)
#reg.fit(X_train, y_train) 
#y_pred = reg.predict(X_test)
#mean_squared_error(y_test, y_pred)  

In [ ]:
#mean_squared_error(y_test, y_pred, multioutput='raw_values')

-2. Cross validation

> RidgeCV implements ridge regression with built-in cross-validation of
> the alpha parameter. The object works in the same way as GridSearchCV
> except that it defaults to Generalized Cross-Validation (GCV), an
> efficient form of leave-one-out cross-validation.

RidgeCV(alphas=(0.1, 1.0, 10.0), fit_intercept=True, normalize=False, scoring=None, cv=None, gcv_mode=None, store_cv_values=False)

cv:
    None, to use the efficient Leave-One-Out cross-validation
    
    integer, to specify the number of folds.

In [ ]:
#reg = linear_model.RidgeCV(alphas = [0.01, 0.1, 1.0, 10])
#reg.fit(X_train, y_train)
#reg.alpha_   

In [ ]:
#reg = linear_model.RidgeCV(alphas = [2.0, 5., 10, 20., 50.])
#reg.fit(X_train, y_train)
#reg.alpha_   

In [ ]:
#reg = linear_model.RidgeCV(alphas = [6., 8., 10., 12., 14., 17.], fit_intercept=True, \
 #                          normalize=False, scoring=None, cv=8, gcv_mode=None, store_cv_values=False)
#reg.fit(X_train, y_train)

In [ ]:
#print(reg.alpha_)
#y_pred = reg.predict(X_test)
#y_pred = np.expm1(y_pred)

In [ ]:
#output = pd.DataFrame(test['Id'])

In [ ]:
#output['SalePrice'] = y_pred

In [ ]:
#output.to_csv('RidgeRegression_firsttry_12232016.csv', index=False)

In [ ]:
# We use the base estimator LassoCV since the L1 norm promotes sparsity of features.
clf = LassoCV()

# Set a minimum threshold, using default threshold 1e-100 ~ 1e-4 first
sfm = SelectFromModel(clf, threshold=1e-100)
sfm.fit(X_train, y_train)
X_transform = sfm.transform(X_train)
n_features = X_transform.shape[1]
print(n_features)

In [ ]:
print(n_features)
print(X_train.shape)
X_index = np.linspace(0, 287, 288)
xv, yv = np.meshgrid(X_index, range(1))
print(xv.shape)
selected_features = sfm.transform(xv)
print(selected_features)

In [ ]:
X_selected_tr = X_train.as_matrix()
print(X_selected_tr.shape)
X_selected_tr = X_selected_tr[:, [3, 5, 6, 8, 22, 24, 26, 27, 28]]

In [ ]:
X_transform - X_selected_tr #Same even though features return type is not integer

In [ ]:


# Reset the threshold till the number of features equals two.
# Note that the attribute can be set directly instead of repeatedly
# fitting the metatransformer.
#while n_features > 2:
#    sfm.threshold += 0.1
#    X_transform = sfm.transform(X)
#    n_features = X_transform.shape[1]

# Plot the selected two features from X.
import matplotlib.pyplot as plt
plt.title(
    "Feature selected from LASSO "
    "threshold %0.3f." % sfm.threshold)
feature1 = X_transform[:, 3]
#feature2 = X_transform[:, 1]
plt.plot(feature1, 'r.')
plt.xlabel("Feature number 1")
plt.ylabel("Feature value")
#plt.ylim([np.min(feature2), np.max(feature2)])
#plt.show()

In [ ]:
from sklearn.linear_model import LassoCV
reg = linear_model.LassoCV(alphas = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2])
reg.fit(X_transform, y_train)
print(reg.alpha_)

In [ ]:
X_test_selected = X_test.as_matrix()[:, [3, 5, 6, 8, 22, 24, 26, 27, 28]]
#y_pred = reg.predict(X_test_selected)
#y_pred = np.expm1(y_pred)

In [ ]:
#output = pd.DataFrame(test['Id'])
#output['SalePrice'] = y_pred
#output.to_csv('Lasso_selected9_03132017.csv', index=False)

In [ ]:
import xgboost as xgb


regr = xgb.XGBRegressor(
                 colsample_bytree=1,
                 gamma=0.0,
                 learning_rate=0.7,
                 max_depth=7,
                 min_child_weight=1.5,
                 #n_estimators=7429,                                                                  
                 reg_alpha=0.6,
                 reg_lambda=0.6,
                 subsample=0.5,
                 #seed=42,
                 silent=1)

regr.fit(X_transform, y_train)

y_pred = regr.predict(X_test_selected)
y_pred = np.expm1(y_pred)

output = pd.DataFrame(test['Id'])
output['SalePrice'] = y_pred
output.to_csv('xgboost_selested9features.csv', index=False)

In [ ]:
xgboost parameters from 
https://www.kaggle.com/zenstat/house-prices-advanced-regression-techniques/xgboost-lasso-copied/code